In [1]:
import boto3
import sagemaker
import os
from sagemaker.amazon.amazon_estimator import get_image_uri

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()

container = get_image_uri(boto3.Session().region_name, 'image-classification')

In [3]:
prefix = 'capstone_data'
s3_dataPath = 's3://{}/{}'.format(bucket, prefix)

train_location = os.path.join(s3_dataPath, 'train')
val_location = os.path.join(s3_dataPath, 'val')
test_location = os.path.join(s3_dataPath, 'test')

s3_lstPath = 's3://{}/lst_files'.format(bucket)

train_lst = os.path.join(s3_lstPath, 'training.lst')
val_lst = os.path.join(s3_lstPath, 'validation.lst')

In [4]:
train_location

's3://sagemaker-us-east-2-991170486756/capstone_data/train'

# AWS Image Classifier

We will be using the HyperParameter Tuning system of AWS, so this process is more complicated than usual. However, it will yield the best results by finding the best model out of a random set of models.

## Constructing the AWS Estimator

In [29]:
ResNet = sagemaker.estimator.Estimator(role=role, 
                                       sagemaker_session=sagemaker_session, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.p2.xlarge',
                                       output_path="s3://{}/{}/output".format(bucket, 'ResNet'), 
                                       image_name=container)

## Hyperparamaters

You can read the documentation for all the Hyperparameters of the Image Classification Algorithm <a href=https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html>here</a>.<br>
<ul>
    <li><b>(Required) num_classes</b>: This is the number of output classes for the new dataset. Ours will be 2 for normal or pneumonia</li>
    <li><b>(Required) num_training_samples</b>: This is the total number of training samples. The training set has 5,216 images in total.</li>
    <li><b>num_layers</b>: The number of layers (depth) for the network. We use will 152 layers</li>
    <li><b>image_shape</b>: The input image dimensions,'num_channels, height, width', for the network. It should be no larger than the actual image size. The number of channels should be same as the actual image.</li>
    <li><b>mini_batch_size</b>: The number of training samples used for each mini batch. In distributed training, the number of training samples used per batch will be N * mini_batch_size where N is the number of hosts on which training is run.</li>
    <li><b>epochs</b>: Number of training epochs.</li>
    <li><b>learning_rate</b>: Learning rate for training.</li>
    <li><b>precision_dtype</b>: Training datatype precision (default: float32). If set to 'float16', the training will be done in mixed_precision mode and will be faster than float32 mode</li>
</ul>

In [5]:
ResNet.set_hyperparameters(num_layers=152, #152 layers for ResNet-152
                           image_shape= "3,224,224", #The dimensions of our cleaned images
                           mini_batch_size=32, #Each batch will have 64 images
                           epochs=10, #10 epochs or training iterations through the training set
                           learning_rate = 0.01, #Learning rate used for the optimizer of the model which is sgd
                           precision_dtype = 'float32',
                           num_training_samples=5215,
                           num_classes=2
                          )

## Hyperparameter Tuning
I will tune the model by some of the embedded Image Classifier hyperparameters, you can read more <a href=https://docs.aws.amazon.com/sagemaker/latest/dg/IC-tuning.html>here</a>. The goal would be to select the best model that is chosen by the highest validation accuracy.

In [14]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner, CategoricalParameter

ResNet_hyperparameter_tuner = HyperparameterTuner(estimator=ResNet, 
                                                  objective_metric_name='validation:accuracy', 
                                                  objective_type='Maximize', 
                                                  max_jobs=5, #Going to run 20 different models
                                                  max_parallel_jobs=1, #Going to train 3 models at the same time
                                                  hyperparameter_ranges={
                                                      #'mini_batch_size':IntegerParameter(64, 128),
                                                      'learning_rate':ContinuousParameter(0.01, 0.1, scaling_type='Logarithmic'),
                                                      'optimizer': CategoricalParameter(['sgd', 'adam', 
                                                                                              'rmsprop', 'nag'])
                                                  })

## Load the data

In [15]:
train_data = sagemaker.session.s3_input(train_location, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(val_location, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')
train_data_lst = sagemaker.session.s3_input(train_lst, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data_lst = sagemaker.session.s3_input(val_lst, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')

data_channels = {'train':train_data, 'validation': validation_data, 
                 'train_lst':train_data_lst, 'validation_lst':validation_data_lst}
    

## Fitting the Hyperparameter Tuning job

Here is where AWS will train multiple training jobs to find the best model out of a number of models.

In [17]:
ResNet_hyperparameter_tuner.fit(data_channels)

In [ ]:
ResNet_hyperparameter_tuner.wait()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

After training numerous models, I will find the one that provided the best training job. <br><br>
Which is here below:

In [27]:
ResNet_hyperparameter_tuner.best_training_job()

'image-classification-200112-2133-002-5c8e0b5a'

I am going to create an AWS <a href=https://sagemaker.readthedocs.io/en/stable/estimators.html>Estimator</a> that uses the training job's model artifacts from above

In [28]:
aws_estimator = sagemaker.estimator.Estimator.attach(training_job_name='image-classification-200112-2133-002-5c8e0b5a')

2020-01-12 23:14:27 Starting - Preparing the instances for training
2020-01-12 23:14:27 Downloading - Downloading input data
2020-01-12 23:14:27 Training - Training image download completed. Training in progress.
2020-01-12 23:14:27 Uploading - Uploading generated training model
2020-01-12 23:14:27 Completed - Training job completedDocker entrypoint called with argument(s): train
[01/12/2020 22:28:25 INFO 139726532032320] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[01/12/2020 22:28:25 INFO 139726532032320] Merging with provided configuration from /opt/ml/input/config/hyperparameters.json: {u

# Constucting the ResNeXt 101-32x8d